In [1]:
# https://langchain-ai.github.io/langgraph/concepts/memory/
"""
Concept: 
- https://langchain-ai.github.io/langgraph/concepts/memory/

Hot Path:
- https://github.com/langchain-ai/memory-agent

"""

In [4]:
import uuid

from langgraph.store.memory import InMemoryStore
from langchain.embeddings import init_embeddings

## KV Search

In [2]:
in_mem_store = InMemoryStore()
user_id = "1"
namespace = (user_id, "memories")

In [5]:
memory_id = str(uuid.uuid4())
memory = {"food_preference": "I like pizza"}
in_mem_store.put(namespace, memory_id, memory)

In [6]:
in_mem_store.search(namespace)

[Item(namespace=['1', 'memories'], key='b8d671b5-f0c2-4e7d-a6a6-2081548fee11', value={'food_preference': 'I like pizza'}, created_at='2025-08-23T04:57:15.020107+00:00', updated_at='2025-08-23T04:57:15.020114+00:00', score=None)]

## Semantic Search

In [7]:
embed = init_embeddings("openai:text-embedding-3-small")
embed

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x1163c9a90>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x1163ca3c0>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [18]:
semantic_store = InMemoryStore(
    index={
        "embed": embed,
        "dims": 1536,
        "fields": ["food_preference", "$"], 
    }
)

In [23]:
user_id = "101"
namespace = (user_id, "memories")
context = "Discussing dinner plans"

semantic_store.put(
    namespace,
    str(uuid.uuid4()),
    {
        "context": context,
        "food_preference": "I like to eat suishi in some restaurants.",
    },
    index=["food_preference"]
)

semantic_store.put(
    namespace,
    str(uuid.uuid4()),
    {
        "context": context,
        "food_preference": "I don't like to eat pizza.",
    },
    index=["food_preference"]
)

semantic_store.put(
    namespace,
    str(uuid.uuid4()),
    {
        "context": context,
        "food_preference": "I am a software engineer",
    },
    index=["food_preference"]
)

In [24]:
semantic_store.search(
    namespace,
    query="What does the user like to eat?",
    limit=3  # Return top 2 matches
)

[Item(namespace=['101', 'memories'], key='5fc0a918-f162-404b-9f34-84c78a171549', value={'context': 'Discussing dinner plans', 'food_preference': 'I like to eat suishi in some restaurants.'}, created_at='2025-08-23T05:24:28.422353+00:00', updated_at='2025-08-23T05:24:28.422367+00:00', score=0.41829255641859375),
 Item(namespace=['101', 'memories'], key='d506d7b2-e387-459a-a9f2-92dcd079c9aa', value={'context': 'Discussing dinner plans', 'food_preference': "I don't like to eat pizza."}, created_at='2025-08-23T05:24:28.722626+00:00', updated_at='2025-08-23T05:24:28.722637+00:00', score=0.34318893647449766),
 Item(namespace=['101', 'memories'], key='27c3b2a4-727e-454a-856f-304b486d8d03', value={'context': 'Discussing dinner plans', 'food_preference': 'I am a software engineer'}, created_at='2025-08-23T05:24:29.439699+00:00', updated_at='2025-08-23T05:24:29.439711+00:00', score=0.12096475421027239)]

In [25]:
semantic_store.put(
    namespace,
    str(uuid.uuid4()),
    {
        "context": context,
        "food_preference": "I really love In-N-Out!!",
    },
    index=["food_preference"]
)

In [26]:
semantic_store.search(
    namespace,
    query="What does the user like to eat?",
    limit=3  # Return top 3 matches
)

[Item(namespace=['101', 'memories'], key='5fc0a918-f162-404b-9f34-84c78a171549', value={'context': 'Discussing dinner plans', 'food_preference': 'I like to eat suishi in some restaurants.'}, created_at='2025-08-23T05:24:28.422353+00:00', updated_at='2025-08-23T05:24:28.422367+00:00', score=0.41820825432270375),
 Item(namespace=['101', 'memories'], key='d506d7b2-e387-459a-a9f2-92dcd079c9aa', value={'context': 'Discussing dinner plans', 'food_preference': "I don't like to eat pizza."}, created_at='2025-08-23T05:24:28.722626+00:00', updated_at='2025-08-23T05:24:28.722637+00:00', score=0.34317582062748114),
 Item(namespace=['101', 'memories'], key='6f8b46b5-d192-44eb-90d4-a0d0b5eb99b1', value={'context': 'Discussing dinner plans', 'food_preference': 'I really love In-N-Out!!'}, created_at='2025-08-23T05:24:35.993076+00:00', updated_at='2025-08-23T05:24:35.993090+00:00', score=0.2719306804324292)]

## Shared Memory between Tasks in Workflow

In [38]:
import asyncio
from datetime import datetime
from typing import List, TypedDict

from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model


# Initialize the language model to be used for memory extraction
llm = init_chat_model("openai:gpt-4.1")

SYSTEM_PROMPT = """You are a helpful and friendly chatbot. \
Try to answer the question give the memories from the users! \
If you't don't have any related information, just said you't don't know.
{memory_text}

System Time: {time}"""


store = InMemoryStore(
    index={
        "embed": init_embeddings("openai:text-embedding-3-small"),
        "dims": 1536,
        "fields": ["food_preference", "$"], 
    }
)

app_id = "my_app_101"
namespace = (app_id, "memories")
context = "Discussing dinner plans"

preference_memories = [
    "Roger likes to eat suishi in Tokyo.",
    "Roger likes in-n-out burgers in California.",
    "Cindy feels Mexico food is delicious.",
    "Cindy doesn't like fast food",
]

for memory_text in preference_memories:
    store.put(
        namespace,
        str(uuid.uuid4()),
        {
            "context": context,
            "food_preference": memory_text,
        },
        index=["food_preference"]
    )


@task
async def ask_question(question: str) -> str:
    memories = store.search(
        namespace,
        query=question,
        limit=2  # Return top 2 matches
    )
    memory_text =  "\n".join(
        f"[{mem.key}]: {mem.value} (similarity: {mem.score})" 
        for mem in memories
    )
    if memory_text:
        memory_text = f"""
        <memories>
        {memory_text}
        </memories>"""

    sys_prompt_text = SYSTEM_PROMPT.format(
        memory_text=memory_text,
        time=datetime.now().isoformat(),
    )
    print(f"question: {question}")
    print(f"sys_prompt_text: {sys_prompt_text}")
    print("====\n\n")

    response = await llm.ainvoke([
        {"role": "system", "content": sys_prompt_text}, 
        {"role": "user", "content": question}, 
    ])
    return response.content


@task
async def store_memory(memory_text: str) -> str:
    store.put(
        namespace,
        str(uuid.uuid4()),
        {
            "context": context,
            "food_preference": memory_text,
        },
        index=["food_preference"]
    )
    return f"Done storing memory: {memory_text}"


class ChatState(TypedDict):
    prev_questions: List[str]
    new_memories: List[str]
    post_questions: List[str]


@entrypoint(checkpointer=InMemorySaver())
async def workflow(s: ChatState) -> list[str]:
    responses = []
    tasks = [
        ask_question(q) for q in s["prev_questions"]
    ] + [
        store_memory(m) for m in s["new_memories"]
    ]

    if tasks:
        for future in asyncio.as_completed(tasks):
            response_text = await future
            print(response_text)
            print("====\n\n")
            responses.append(response_text)

    tasks = [
        ask_question(q) for q in s["post_questions"]
    ]
    if tasks:
        for future in asyncio.as_completed(tasks):
            response_text = await future
            print(response_text)
            print("====\n\n")
            responses.append(response_text)

    return responses

In [39]:
# Call the entrypoint
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

responses = await workflow.ainvoke(
    ChatState(
        prev_questions=[
            f"What are food preferences for {name}"
            for name in ["Roger", "Cindy", "Michael"]
        ],
        new_memories=[
            "Mary likes to eat pizza!"
        ],
        post_questions=[
            "What about Mary's favorite foods?"
        ],
    ), 
    config,
)

question: What are food preferences for Roger
sys_prompt_text: You are a helpful and friendly chatbot. Try to answer the question give the memories from the users! If you't don't have any related information, just said you't don't know.

        <memories>
        [3abbeb40-ced5-4e5a-8968-0c895c6f0dab]: {'context': 'Discussing dinner plans', 'food_preference': 'Roger likes to eat suishi in Tokyo.'} (similarity: 0.46379459224586417)
[e5fc332c-5a63-4893-a03f-f3daf81b2a2e]: {'context': 'Discussing dinner plans', 'food_preference': 'Roger likes in-n-out burgers in California.'} (similarity: 0.42955565157667663)
        </memories>

System Time: 2025-08-24T17:23:48.520335
====


question: What are food preferences for Cindy
sys_prompt_text: You are a helpful and friendly chatbot. Try to answer the question give the memories from the users! If you't don't have any related information, just said you't don't know.

        <memories>
        [e7834af3-08c9-4c80-a0d7-06815ed522d3]: {'context': 